In [2]:
%pip install ollama
%pip install numpy


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/2b/86/d019fb60a9d0f1d4cf04b014fe88a9135090adfadcc31c1fadbb071d7fa7/numpy-2.2.2-cp312-cp312-macosx_14_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 115.8 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 178.0 kB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
from ollama import chat
from IPython.display import Markdown, display
from numpy import arange

rule = '''
你的任务是做 javascript 代码质量检查，下面会定义一些检查规则，并列举正确案例和错误案例供你参考。
规则由规则描述、规则说明、错误案例、正确案例，四部分构成。

1. 规则描述以 Markdown H1 标题呈现。
2. 规则说明以 H2 标题呈现，可以省略。
3. 错误案例和正确案例是示例代码，以H2标题呈现，代码中的注释部分会对代码中合理和不合理的地方做出解释。

后续将会告诉你要检查的代码，你的目的是将不符合规则要求的代码输出出来，给出错误原因，并给出改进意见。

最后将对代码的检查结果以下面定义的 JSON 格式输出：
```
Array<{
	line: number, // 出错的代码行数
	reason: string, // 违反的规则
	fix: string, // 修改建议	
}>;
```

以下是定义的规则：

# 在大于 5 行的函数里，或者全局代码中，不要使用单字命名的变量。
## 错误案例
```
var l = data.length;
b = l + a; 
```

## 正确案例
```
data.map(d => d.length);
```

# 应该有且只有方法和函数应以动词开头

## 规则说明
函数的命名需要体现内部工作，应该以不包含时态的动词开头。
值变量以动词命名，则会容易让人误解为是一个匿名函数。

以下函数的命名可以不遵守此规则：
- 回调函数
- 生命周期函数
- 框架级别函数
- getter/setter 

## 错误案例
```
// 以名词开头，看不明白什么有什么功能
function option() {}

// 时态不对
function updatedTime() {}
```

## 正确案例
```
function selectOption() {}
function updateTime() {}

class App {
    // 因为是 getter 函数，不需要遵守此规则
    get name() { }
    // 因为是生命周期函数，所以不需要遵守此规则
    componentDidMount() {}
    // 因为是点击事件回调函数，不需要遵守此规则
    onModalClick() { }
}
```
'''

test = '''
let a = 1;
[].map(b => a+b);
let doList = [];
class Circle{
    onMounted(){}
    makedName(b) {
        return <span>{b}</span>;
    }
    renderList() {
        var a = this.data;
        var l = a.map(b => {
            return this.makedName(b);
        });
        l = l.filter(Boolean);
        return l;
    }
    render() {
        return <div>this.renderList()</div>;
    }
}
var rect = new Circle();
'''
messages = [
  {
    'role': 'user',
    'content': rule,
  },
  {
    'role': 'user',
    'content': test,
  }
]

def deleteThink(original_string):
    while '<think>' in original_string:
      start_index = original_string.find('<think>')
      end_index = original_string.find('</think>') + len('</think>')
      # 切片操作，将匹配到的部分删除
      original_string = original_string[:start_index] + original_string[end_index:]
    return original_string

for temp in range(0, 2):
  response = chat('deepseek-r1:32b', messages=messages)
  content = deleteThink(response['message']['content'])
  # display(Markdown(content))
  print(f'run with temperature {temp}')
  print(content)

run with temperature 0


```json
[
    {
        "line": 1,
        "reason": "在全局代码中使用了单字命名的变量",
        "fix": "建议改为具有描述性的名称，例如 'initialValue' 或 'data'"
    },
    {
        "line": 7,
        "reason": "在大于5行的函数里使用了单字命名的变量",
        "fix": "建议改为更具描述性的名称，如 'dataset' 或 'currentData'"
    },
    {
        "line": 8,
        "reason": "在大于5行的函数里使用了单字命名的变量",
        "fix": "建议改为更明确的名称，例如 'items' 或 'elements'"
    }
]
```
run with temperature 1


```json
[
    {
        "line": 1,
        "reason": "在大于5行的函数里，或者全局代码中，不要使用单字命名的变量。",
        "fix": "将 'a' 改为更具描述性的名称，例如 'initialValue'。"
    },
    {
        "line": 2,
        "reason": "在大于5行的函数里，或者全局代码中，不要使用单字命名的变量。",
        "fix": "将 'b' 改为更具描述性的名称，例如 'item' 或者根据上下文更合适的名称。"
    },
    {
        "line": 7,
        "reason": "在大于5行的函数里，或者全局代码中，不要使用单字命名的变量。",
        "fix": "将 'a' 改为更具描述性的名称，例如 'dataList' 或者根据上下文更合适的名称。"
    },
    {
        "line": 8,
        "reason": "在大于5行的函数里，或者全局代码中，不要使用单字命名的变量。",
        "fix": "将 'l' 改为更具描述性的名称，例如 'm